In [1]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
DS_NAME = 'gaussian'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/rf_{DS_NAME}.pkl')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)[:1000]
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)[:1000]
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)[:1000]

In [3]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target, n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target, n_shadow_models=1)
aloa = AloaPrivacyAttack(target, n_shadow_models=1, n_noise_samples_fit=200,
                         shadow_test_size=0.3, undersample_attack_dataset=True)
attacks = [mia, label_only, aloa]

In [4]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [5]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

Each shadow model uses 100.000 % of the data


100%|██████████| 1000/1000 [00:14<00:00, 68.78it/s]


Each shadow model uses 100.000 % of the data


100%|██████████| 600/600 [00:08<00:00, 68.27it/s]


In [6]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


100%|██████████| 2000/2000 [00:34<00:00, 57.16it/s]

{'mia_attack': {'classification_report': {'IN': {'precision': 0.5145228215767634, 'recall': 0.992, 'f1-score': 0.6775956284153004, 'support': 1000}, 'OUT': {'precision': 0.8888888888888888, 'recall': 0.064, 'f1-score': 0.11940298507462685, 'support': 1000}, 'accuracy': 0.528, 'macro avg': {'precision': 0.7017058552328261, 'recall': 0.528, 'f1-score': 0.39849930674496364, 'support': 2000}, 'weighted avg': {'precision': 0.7017058552328261, 'recall': 0.528, 'f1-score': 0.39849930674496364, 'support': 2000}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1000}, 'OUT': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1000}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2000}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2000}}}, 'aloa_attack': {'classification_report': {'IN': {'precis

In [7]:
results['mia_attack']

{'classification_report': {'IN': {'precision': 0.5145228215767634,
   'recall': 0.992,
   'f1-score': 0.6775956284153004,
   'support': 1000},
  'OUT': {'precision': 0.8888888888888888,
   'recall': 0.064,
   'f1-score': 0.11940298507462685,
   'support': 1000},
  'accuracy': 0.528,
  'macro avg': {'precision': 0.7017058552328261,
   'recall': 0.528,
   'f1-score': 0.39849930674496364,
   'support': 2000},
  'weighted avg': {'precision': 0.7017058552328261,
   'recall': 0.528,
   'f1-score': 0.39849930674496364,
   'support': 2000}}}

In [8]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1000},
 'OUT': {'precision': 0.5,
  'recall': 1.0,
  'f1-score': 0.6666666666666666,
  'support': 1000},
 'accuracy': 0.5,
 'macro avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000},
 'weighted avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000}}

In [9]:
aloa.attack_model.threshold

0.53